In [2]:
import pandas as pd
import numpy as np
import requests
import json
import ast
from glob import glob
from time import time
import gc

In [3]:
def join_files(files):
    tudo = ''
    for i, f in enumerate(files):
        with open(f) as file:
            t = pd.read_csv(file, usecols=['date','review'])
            if i == 0:
                tudo = t
            else:
                tudo = pd.concat([tudo, t], axis=0)
    return tudo

def request_api(sentences, url = 'http://localhost:8000/v1/all'):
    r = requests.post(url, json={"sentences":sentences})
    r.encoding = 'utf-8'
    return json.loads(r.text)

def request_from_big_file(file, chunksize):
    tudo = []
    url = 'http://localhost:8000/v1/clean_text'
    for data in pd.read_csv(file, usecols=['date','review'], chunksize=50000):   
        sentences = list(data.review.values)
        resposta = request_api(sentences)['result']        
        clean = request_api(sentences, url)['result']
        dates = data.date.values
        for i, value in enumerate(clean):
            resposta[i]['clean'] = value
            resposta[i]['date'] = dates[i]
        tudo += resposta
    
    df = pd.DataFrame(tudo)
    df.to_csv('processado.csv', index=False)
    df.to_json('all_reviews.json', force_ascii=False, orient='records')

In [4]:
# Descomentar linhas abaixo para gerar o arquivo com os dados a ser analisados
files = glob('../../data_extraction/csv/app/*.csv')
data = join_files(files)
data = data.sort_values(by='date')
data['datetime'] = pd.to_datetime(data['date'])
start_date = '01-01-2019'
end_date = '30-04-2020'
mask = (data['datetime'] > start_date) & (data['datetime'] <= end_date)
data = data[mask]
data = data.fillna('')
data = data.drop_duplicates().reset_index(drop=True)
del data['datetime']
data.to_csv('all_reviews.csv',index=False)
print(data.shape)
del data
gc.collect()

(528942, 2)


0

In [5]:
inicio = time()
#Processo necessário para não gerar estouro de memória
request_from_big_file('all_reviews.csv',50000)
print('Processo finalizado em', time()-inicio, 'seg')

Processo finalizado em 198.4850401878357 seg


data_full.to_csv('dataset.csv',index=False)

data_full.to_json('data.json', force_ascii=False, orient='records')

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(analise['result'], f, ensure_ascii=False, indent=4)

loja = data_full[['date','rating','review','store','version']]

loja.to_csv('dados.csv',index=False)

d = pd.read_csv('dataset.csv', usecols=['texto','clean','key_words','sent'])
d = d.fillna('')
d.key_words = d.key_words.apply(ast.literal_eval)
res_json = d.to_dict(orient='records')

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(res_json, f, ensure_ascii=False, indent=4)

d.to_json('data.json', force_ascii=False, orient='records')